In [1]:
import os
import re
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
#from imblearn.over_sampling import SMOTE

%matplotlib inline


## Functions preparation
### Data cleaning

In [2]:
def clean(data_raw):
    """
    clean the input data for future use
    
    Parameters:
    -----------
    data: pd.DataFrame()
    
    Returns:
    result pd.DataFrame() after cleaning
    
    Date: 1/21/2020
    """
    
    # copy data
    data = data_raw.copy()
    
    # convert na to 0 for following columns
    data["Comments_count"].fillna(0, inplace=True)
    data["Bookmarks_count"].fillna(0, inplace=True)
    data["Shares_count"].fillna(0, inplace=True)
    
    # if End_date is NaN, use Posted_date values
    data["End_date"].fillna(data['Posted_date'], inplace=True)

    # convert End_date and Posted_date string to datatime format
    for key in ["Posted", "End"]:
        data[key+'_date']= pd.to_datetime(data[key+'_date'])
        
    # if End_date is earlier than Posted_date, convert it to Posted_date
    end_before_post = data["End_date"] < data["Posted_date"]
    data.loc[end_before_post, "End_date"] = data.loc[end_before_post, "Posted_date"]
    
    for key in ["Posted", "End"]:
        data[key+'_year'] = pd.DatetimeIndex(data[key+'_date']).year
        data[key+'_month'] = pd.DatetimeIndex(data[key+'_date']).month
        data[key+'_day'] = pd.DatetimeIndex(data[key+'_date']).day
        data[key+'_weekday'] = data[key+'_date'].dt.weekday
            
    # extracting xx% off from Description
    data['discount_per_cent'] = data["Description"].str.extract(r'(?P<discount>\d+)[%]')
    data['discount_per_cent'] = data['discount_per_cent'].astype('float32')
    #print(data['discount_per_cent'].head())
    data.drop(["Title", "Description"], axis=1, inplace=True)
    
    # if discount value is not found, the discount_per_cent is nan. Drop such data.
    data.dropna(inplace=True)

    return data

### Calculate the holidays in US


In [3]:
import holidays
def get_holidays(yearrange = None):
    """
    Get the pd.DataFrame containing all the holidays date and names for the given years
    Parameters:
    -----------
    yearrange: list of years to be included in the result
    
    Returns:
    --------
    pd.DataFrame of holidays' date and names
    """
    if yearrange is None:
        return None
    
    holidaydict = {"date": [], "name": []}
    for date, name in holidays.US(years=yearrange).items():
        if "Observed" in name:
            continue
        holidaydict["date"].append(date)
        holidaydict["name"].append(name.replace(',', '').replace('\'','').replace('.','').replace(' ', '_'))
        #print(date, name)
    pd_holiday = pd.DataFrame(holidaydict)
    pd_holiday["date"] = pd.to_datetime(pd_holiday["date"])
    return pd_holiday

In [4]:
def ndays_from_holiday(dfhday, hname, test_date):
    """
    In the given holidays (dfhday), find the number of days away from the given holiday (hname).
    Parameters:
    -----------
    dfhday: pd.DataFrame() with two columns "date", "name" of the holidays
    hname: string, given holiday name
    test_date: datetime, date to be calculated for the ndays to holidays
    
    Returns:
    number of days away from the holidy converted to a score [0, 1] exp(-x**2/21**2), 
    where 21 (3 weeks) is chosen as sigma of the gaussian.
    """
    hdaynamed = dfhday[ dfhday["name"] == hname ]
    if hdaynamed is None or len(hdaynamed)==0:
        print("ERROR: holiday: %s, is not found. Return NaN."%hname)
        return np.nan
    ndays = (test_date - hdaynamed["date"]).dt.days.abs().min()
    ndexp = np.exp(-ndays**2/(21**2)) # sigma is 21
    
    #return ndays
    return ndexp


### Make Figures


In [5]:
def plot_count(data_clean = None, by = "Posted_year", column = "Posted_month", title = "Year"):
    """
    Plot the counting for column separated by --by-- variable
    """
    for var in range(data_clean[by].min(), data_clean[by].max()+1):
        data_slice = data_clean[data_clean[by] == var].copy()
        plt.figure()
        ax = sns.countplot(x= column, data= data_slice)
        ax.set_title(title+" = %d"%var)

### Build Features

In [6]:
from datetime import timedelta, date
def features(data):
    """
    calculate features as X as well as prediction for having any discount or not in the next, say 3, days!
    """
    feature_dict = {
        "Date":[],
        "ndays_of_deal":[], # maximum of how many days does the current day's deal lasted
        "avg_comments":[], # average comments
        "avg_bookmarks":[], # average bookmarks
        "avg_shares":[], # average shares
        "year": [], "month": [], "day":[], "weekday": [],
        "discount_today":[], # 0: no discount today, 1: with discount today
        "avg_discount":[], # average discount of today's deals
        "Y_discount_1d":[], # discount or not in the next 1 day
        "Y_avg_discount_1d":[], # average amount of discount in the next 1 day
        "Y_discount_3d":[], # discount or not in the next 3 days
        "Y_avg_discount_3d":[], # average amount of discount in the next 3 days
        "Y_discount_7d":[], # discount or not in the next 7 days
        "Y_avg_discount_7d":[], # average amount of discount in the next 7 days
        "Y_discount_14d":[], # discount or not in the next 14 days
        "Y_avg_discount_14d":[], # average amount of discount in the next 14 days
    }
    # lagging variables
    # for example: discount_past1day meaning:
    #   in the past 1 day (yesterday) whether or not there is discount, 0 or 1
    #   do it for the past 1 -- 7 days
    feature_dict.update({"discount_past%dday"%i:[] for i in range(1,8)})
    # another example: discount_min_past15day meaning:
    #   in the past 15 days the mininum discount of one of the days
    #   do it for maximum/mean as well as for 30 days
    feature_dict.update({"discount_%s_past%dday"%(s,i):[] for s in ['min', 'max', 'mean'] for i in [15, 30]})
    
    start_date = data["Posted_date"].min()
    ndays = int((data["Posted_date"].max() - start_date).days) + 1 - 3
    for jd in range(ndays):
        today = start_date + timedelta(jd)
        data_today = data[data["Posted_date"]==today]
        feature_dict["year"].append(today.year)
        feature_dict["month"].append(today.month)
        feature_dict["day"].append(today.day)
        feature_dict["weekday"].append(today.weekday())
        for ndp in [1, 3,7,14]:
            data_nextnd = data[(data["Posted_date"]>today) & (data["Posted_date"]<=today+timedelta(ndp))]
            
            if data_nextnd is None or len(data_nextnd)==0:
                feature_dict["Y_discount_%dd"%ndp].append(0)
                feature_dict["Y_avg_discount_%dd"%ndp].append(0)
            else:
                feature_dict["Y_discount_%dd"%ndp].append(1)
                # calculate the average discount in the future n-days
                # use ranges [0, 4]% -> 0; [5, 14]% -> 10%, [15, 24]% -> 20%
                avg_f = data_nextnd["discount_per_cent"].mean()
                avg_d = 0
                for i in range(11):
                    if (avg_f >= i*10 - 5) and (avg_f < i*10 + 5):
                        avg_d = i*10
                        break
                feature_dict["Y_avg_discount_%dd"%ndp].append(avg_d)
            
        ndays_deal, avg_disc, avg_comm, avg_book, avg_shar = 0, 0., 0., 0., 0.
        if (data_today is not None) and (len(data_today)>0):
            ndays_deal = 1 + int((data_today["End_date"].max() - data_today["Posted_date"].iloc[0]).days) 
            avg_disc = data_today["discount_per_cent"].mean() 
            avg_comm = data_today["Comments_count"].mean() 
            avg_book = data_today["Bookmarks_count"].mean() 
            avg_shar = data_today["Shares_count"].mean()
            
        # lagging variables
        for nd_past in range(1, 8):
            # only when jd >= nd_past, one can calculate the lagging variable using the past
            # nd_past days
            if jd < nd_past:
                feature_dict["discount_past%dday"%nd_past].append(np.nan) # or use nan or 0???
            else:
                #feature_dict["discount_past%dday"%nd_past].append( feature_dict["Y_discount_1d"][jd - nd_past])
                feature_dict["discount_past%dday"%nd_past].append( feature_dict["discount_today"][-nd_past])
        for nd_past in [15, 30]:
            if jd < nd_past:
                for s in ['min', 'max', 'mean']:
                    feature_dict["discount_%s_past%dday"%(s,nd_past)].append(np.nan) # or use 0???
            else:
                #nd_past_discounts = np.array( feature_dict["Y_avg_discount_1d"][jd-nd_past:jd] )
                nd_past_discounts = np.array( feature_dict["avg_discount"][jd-nd_past:jd] )
                feature_dict["discount_min_past%dday"%nd_past].append(nd_past_discounts.min())
                feature_dict["discount_max_past%dday"%nd_past].append(nd_past_discounts.max())
                feature_dict["discount_mean_past%dday"%nd_past].append(nd_past_discounts.mean())
            
 
        feature_dict["Date"].append(today)
        feature_dict["ndays_of_deal"].append(ndays_deal)
        feature_dict["discount_today"].append(0 if avg_disc < 1.e-5 else 1)
        feature_dict["avg_discount"].append(avg_disc)
        feature_dict["avg_comments"].append(avg_comm) 
        feature_dict["avg_bookmarks"].append(avg_book) 
        feature_dict["avg_shares"].append(avg_shar) 
    
    pd_feature = pd.DataFrame(feature_dict)
    pd_feature["Date"] = pd.to_datetime(pd_feature["Date"])

    return pd_feature


## Import the datasets and cleaning

In [7]:
input_filefolder = "Datasets"
input_brand_list = ["Carters", "Oshkosh"]

In [8]:
data_clean_dict = dict()
year_min, year_max = 9999, 0
for brand in input_brand_list:
    filename = input_filefolder+"/"+brand+".csv"
    data_raw = pd.read_csv(filename)
    #print(data_raw.head(2))
    #print(data_raw.columns)
    data_clean = clean(data_raw)
    print(data_clean.head())
    data_clean_dict[brand] = data_clean
    
    # calculate the year range for holiday variables later
    ymin,ymax=data_clean["Posted_year"].min()-1, data_clean["Posted_year"].max()+2
    year_min = ymin if ymin < year_min else year_min
    year_max = ymax if ymax > year_max else year_max

     Brand Posted_date   End_date  Comments_count  Bookmarks_count  \
0  Carters  2020-01-21 2020-01-21             4.0              2.0   
1  Carters  2020-01-21 2020-01-21             0.0              4.0   
2  Carters  2020-01-21 2020-01-21            10.0             55.0   
3  Carters  2020-01-21 2020-01-21            15.0            139.0   
6  Carters  2020-01-21 2020-01-21             1.0             14.0   

   Shares_count  Posted_year  Posted_month  Posted_day  Posted_weekday  \
0           2.0         2020             1          21               1   
1           3.0         2020             1          21               1   
2          30.0         2020             1          21               1   
3          99.0         2020             1          21               1   
6           9.0         2020             1          21               1   

   End_year  End_month  End_day  End_weekday  discount_per_cent  
0      2020          1       21            1               60.0  
1 

### Prepare the holiday list dataframe


In [9]:
# consider holidays from year before and year after, because the closest holiday may not in the
# same year as the date under consideration
hyearrange=range(year_min, year_max)
print("holiday year range:", hyearrange)
pd_holiday = get_holidays(hyearrange)
print(pd_holiday.head())
holidayset = set([h for h in pd_holiday["name"]])
print(holidayset)


holiday year range: range(2010, 2022)
        date                       name
0 2016-01-01              New_Years_Day
1 2016-01-18  Martin_Luther_King_Jr_Day
2 2016-02-15       Washingtons_Birthday
3 2016-05-30               Memorial_Day
4 2016-07-04           Independence_Day
{'Washingtons_Birthday', 'Veterans_Day', 'Memorial_Day', 'Labor_Day', 'Martin_Luther_King_Jr_Day', 'Thanksgiving', 'Independence_Day', 'Christmas_Day', 'New_Years_Day', 'Columbus_Day'}


In [11]:
for brand in input_brand_list:
    data_clean = data_clean_dict[brand]
    print("Brand:%s"%brand)
    data_feature = features(data_clean)
    data_feature_holiday = data_feature[["Date"]].copy()
    for h in holidayset:
        print("holiday: %s"%h)
        data_feature_holiday["nday_away_"+h] = \
        data_feature_holiday.apply(lambda row: ndays_from_holiday(pd_holiday, h, row["Date"]), axis=1)  
    #print(data_feature_holiday.head())
    data_feature_holiday["nday_away_anyholiday"] = \
    data_feature_holiday[["nday_away_"+h for h in holidayset]].max(axis=1)
    data_feature_update = pd.concat([data_feature, data_feature_holiday.drop(["Date"], axis=1)], axis=1)
    data_feature_update.drop(["ndays_of_deal"], axis=1, inplace=True)
    data_feature.dropna(inplace=True)
    fileoutname = input_filefolder+"/"+brand+"_features.csv"
    data_feature_update.to_csv(fileoutname)
    print("Saved:%s\n"%fileoutname)
    #print(data_feature_update.head())
    #print(len(data_feature_update.columns), data_feature_update.columns)
    

Brand:Carters
holiday: Washingtons_Birthday
holiday: Veterans_Day
holiday: Memorial_Day
holiday: Labor_Day
holiday: Martin_Luther_King_Jr_Day
holiday: Thanksgiving
holiday: Independence_Day
holiday: Christmas_Day
holiday: New_Years_Day
holiday: Columbus_Day
Saved:Datasets/Carters_features.csv

Brand:Oshkosh
holiday: Washingtons_Birthday
holiday: Veterans_Day
holiday: Memorial_Day
holiday: Labor_Day
holiday: Martin_Luther_King_Jr_Day
holiday: Thanksgiving
holiday: Independence_Day
holiday: Christmas_Day
holiday: New_Years_Day
holiday: Columbus_Day
Saved:Datasets/Oshkosh_features.csv

